In [2]:
from Organizer import Organizer


Researcher = Organizer()

data = await Researcher.search("lily flower types")

data

lily flower types


[{'title': 'Types of Lilies: 18 Stunning Lily Species for Your Garden', 'description': "Sep 10, 2023—If you're looking for the perfectlilyto grow in your garden, this complete guide by a master gardener for the top 18typesofliliesis\xa0...", 'url': 'https://www.planetnatural.com/types-of-lilies/'}, {'title': 'Types of lilies: varieties & characteristics', 'description': "Typesoflilies: overview of the most popularlilyvarieties · 1. Oriental hybrids · 2. Asiatic hybrids · 3. Tigerlily(Lilium lancifolium) · 4. Turk's caplily(\xa0...", 'url': 'https://plantura.garden/uk/flowers-perennials/lilies/types-of-lilies'}, {'title': '9 Types of Lilies | Petal Talk', 'description': 'Jul 10, 2024—They can be grouped into nine main categories: Asiatic, Oriental, Martagon, American, Longiflorum, Aurelian, Candidum, Interdivisional, and Species.', 'url': 'https://www.1800flowers.com/articles/flower-facts/types-of-lilies'}, {'title': '74 Different Types of Beautiful Lily Varieties', 'description': "Oct 

[{'url': 'https://en.wikipedia.org/wiki/Lilium',
  'metadata': {'title': 'Lilium - Wikipedia',
   'description': '',
   'language': 'en',
   'sourceURL': 'https://en.wikipedia.org/wiki/Lilium'},
  'structured_data': {'full_text': 'Main menu Main menu move to sidebar hide Navigation Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us Contribute HelpLearn to editCommunity portalRecent changesUpload file Search Search Donate Appearance Create account Log in Personal tools Create account Log in Pages for logged out editors learn more ContributionsTalk\n\nMain menu Main menu move to sidebar hide Navigation Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us Contribute HelpLearn to editCommunity portalRecent changesUpload file\n\nMain menu Main menu move to sidebar hide Navigation Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us Contribute HelpLearn to editCommunity portalRecent changesUpload file\n\nMain menu move to sidebar hide N

['/static/images/icons/wikipedia.png',
 '/static/images/mobile/copyright/wikipedia-wordmark-en.svg',
 '/static/images/mobile/copyright/wikipedia-tagline-en.svg',
 '//upload.wikimedia.org/wikipedia/commons/thumb/3/30/Lilium_candidum_1.jpg/220px-Lilium_candidum_1.jpg',
 '//upload.wikimedia.org/wikipedia/commons/thumb/8/8a/OOjs_UI_icon_edit-ltr.svg/15px-OOjs_UI_icon_edit-ltr.svg.png',
 '//upload.wikimedia.org/wikipedia/commons/thumb/a/a4/Lillium_Stamens.jpg/250px-Lillium_Stamens.jpg',
 '//upload.wikimedia.org/wikipedia/commons/thumb/a/a6/Lily%2C_petal.jpg/170px-Lily%2C_petal.jpg',
 '//upload.wikimedia.org/wikipedia/commons/thumb/9/9d/2012-01-04_17-57-36-lys.jpg/170px-2012-01-04_17-57-36-lys.jpg',
 '//upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Lilium_candidum_MHNT.BOT.2011.18.27.jpg/170px-Lilium_candidum_MHNT.BOT.2011.18.27.jpg',
 '//upload.wikimedia.org/wikipedia/commons/thumb/3/32/Lilium_distichum.jpg/120px-Lilium_distichum.jpg',
 '//upload.wikimedia.org/wikipedia/commons/thumb/f/

In [8]:
a = [1,2]

a.extend([3,4])
a

[1, 2, 3, 4]